In [1]:
from llama import LLM

In [7]:
config = {
    "production": {
        "key": "test_token",
        "url": "http://api:8000"
    }
}

In [3]:
from llama import Type, Context

# Input
class Question(Type):
    question: str = Context("question about the function")


# Output
class Answer(Type):
    inputs: list = Context("list of inputs to the function")
    outputs: list = Context("list of outputs from the function")
    description: str = Context("function description in 2 to 5 lines")

In [4]:
class Function(Type):
    name: str = Context("name of the function")
    code: str = Context("text for the python code in the function")

In [5]:
# @title Preprocess data
# @markdown This collects our llama functions into the type format.

import llama
from inspect import getmembers, getsource, isfunction, ismodule, isclass

seen = set()
def get_functions(module):
    functions = set()
    if module not in seen:
        seen.add(module)
        for name, member in getmembers(module):
            if isfunction(member):
                functions.add((name, getsource(member).strip()))
            elif ismodule(member) and member.__name__.startswith("llama"):
                functions.update(get_functions(member))
            elif isclass(member) and hasattr(member, '__module__') and f'{member.__module__}.{member.__qualname__}'.startswith("llama"):
                functions.update(get_functions(member))
    return functions

def make_function(function):
    return Function(
        name=function[0],
        code=function[1]
    )

functions = [make_function(function) for function in get_functions(llama)]

In [8]:
from llama import LLM

llm = LLM(name="generate_docs", config=config)
llm.add_data(functions)
question = 'LLM.__init__'
answer = llm(input=Question(question=question), output_type=Answer)

APIError: API error 500

In [ ]:
SEPARATOR = "\n\n"
def get_function_docs(question, answer):
    docstring = f" # Function:{SEPARATOR}{question}{SEPARATOR}"
    docstring += f"## Inputs:{SEPARATOR}"
    for input in answer.inputs:
        docstring += f"{input}{SEPARATOR}"
    docstring += f"## Outputs:{SEPARATOR}"
    for output in answer.outputs:
        docstring += f"{output}{SEPARATOR}"
    docstring += f"## Description:{SEPARATOR}{answer.description}"
    print(docstring)

get_function_docs(question, answer)